In [ ]:
%pyspark
#All imports go here
from shapely.geometry import shape
from shapely.ops import transform
from shapely.geometry import MultiPolygon
from shapely import wkt
from shapely.ops import transform

from functools import partial
import pyproj

import pyspark.sql.functions as pys
from pyspark.sql.types import DoubleType, StringType, FloatType, IntegerType, BooleanType
from pyspark.sql.functions import udf
from pyspark.sql.functions import sum
from pyspark.sql.functions import col
from pyspark.sql import functions as F

import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
from pyspark.sql.functions import desc as dsc


In [ ]:
%pyspark
fields = sqlContext.read.json("s3n://data-lake-us-east-2-549323063936-validated/NonExploded_New_data_field.json/*.json")
fields = fields.where(col('field.deleted_at').isNull())
fields2 = fields.select('field')


In [ ]:
%pyspark
sampleEvents = sqlContext.read.json("s3n://data-lake-us-east-2-549323063936-validated/NonExploded_New_data_sample_event.json/*.json")


In [ ]:
%pyspark
events = sqlContext.read.json("s3n://data-lake-us-east-2-549323063936-validated/NonExploded_New_data_event.json/*.json")


In [ ]:
%pyspark
fieldEvents = sqlContext.read.json("s3n://data-lake-us-east-2-549323063936-validated/NonExploded_New_data_field_event.json")

In [ ]:
%pyspark
sampleEvents.printSchema()
events.printSchema()
fieldEvents.printSchema()


In [ ]:
%pyspark
sampleEvents.select('sample_event.sample_event_type').show()

In [ ]:
%pyspark
sampleEventFieldIds = sampleEvents.select('sample_event.associations.`agrian.field`').distinct()

In [ ]:
%pyspark
eventFieldIds = events.select('event.associations.`agrian.field`').distinct()

In [ ]:
%pyspark
fieldEventFieldIds = fieldEvents.select('field_event.associations.`agrian.field`').distinct()

In [ ]:
%pyspark
fieldIds = fields.select('field.id').distinct()


In [ ]:
%pyspark
sampleEventFieldIds = sampleEventFieldIds.select(pys.explode('`agrian.field`')).toDF('fieldSEId')
fieldEventFieldIds = fieldEventFieldIds.select(pys.explode('`agrian.field`')).toDF('fieldFEId')
eventFieldIds = eventFieldIds.select(pys.explode('`agrian.field`')).toDF('fieldEId')

In [ ]:
%pyspark
fieldsWithAllData = sampleEventFieldIds.join(fieldEventFieldIds.join(eventFieldIds, eventFieldIds.fieldEId == fieldEventFieldIds.fieldFEId, "inner"), fieldEventFieldIds.fieldFEId == sampleEventFieldIds.fieldSEId, 'inner'  )
# eventsAndFEvents = fieldEventFieldIds.join(eventFieldIds, eventFieldIds.fieldEId == fieldEventFieldIds.fieldFEId, "inner")
fieldsWithAllData.count()

In [ ]:
%pyspark
fields.count()

In [ ]:
%pyspark
fieldsWithAllData.show()


In [ ]:
%pyspark
eventsTemp = events.sort(desc('event.associations.`agrian.field`') , desc('event.created_at')).groupby('event.associations.`agrian.field`').agg(F.collect_list("event.created_at"), F.collect_list("event.json.event_type"), F.collect_list("event"))
# eventsTemp.show()

In [ ]:
%pyspark
fieldEventsTemp = fieldEvents.sort(desc('field_event.associations.`agrian.field`'), desc("field_event.created_at")).groupby('field_event.associations.`agrian.field`').agg(F.collect_list("field_event.created_at"), F.collect_list("field_event.event_type") , F.collect_list("field_event"))
# fieldEventsTemp.show()

In [ ]:
%pyspark
sampleEventsTemp = sampleEvents.sort(desc('sample_event.associations.`agrian.field`'), desc("sample_event.created_at")).groupby('sample_event.associations.`agrian.field`').agg(F.collect_list("sample_event.created_at"), F.collect_list("sample_event.sample_event_type"), F.collect_list("sample_event"))
#sampleEventsTemp.show()

In [ ]:
%pyspark
fieldsMin = fields.select('field.id', 'field.associations.`agrian.grower`').toDF('id', 'grower_id')
fieldsMin = fieldsMin.distinct()
fieldsMin.printSchema()

In [ ]:
%pyspark
joined = fieldsMin.join(sampleEventsTemp, sampleEventsTemp['`agrian.field`'][0] == fieldsMin.id, 'inner' )

In [ ]:
%pyspark
joined = joined.join(fieldEventsTemp, fieldEventsTemp['`agrian.field`'][0] == joined.id, 'inner' )


In [ ]:
%pyspark
joined = joined.join(eventsTemp, eventsTemp['`agrian.field`'][0] == joined.id, 'inner' )


In [ ]:
%pyspark
joined = joined.drop('agrian.field')

In [ ]:
%pyspark
joined.printSchema()

In [ ]:
%pyspark
joined.write.format("json").save("s3n://data-lake-us-east-2-549323063936-validated/timeStampedEventsForFields")

In [ ]:
%pyspark
growers = sqlContext.read.json("s3n://data-lake-us-east-2-549323063936-validated/NonExploded_New_data_grower.json/*.json")


In [ ]:
%pyspark
grower.printSchema()

In [ ]:
%pyspark
growerMin = grower.select('grower.id', 'grower.alternate_ids').toDF('grower_id', 'alternateIds')

In [ ]:
%pyspark
ars = growerMin.select('grower_id', F.explode(growerMin.alternateIds)).toDF('grower_id', 'alternateID')
ars = ars.select('grower_id', 'alternateID.*')
ars = ars.where('id_type == "ars_id"')


In [ ]:
%pyspark
ars = ars.toDF('growerArsId', 'alternate_id', 'alteranate_ars_id', 'id_type')

In [ ]:
%pyspark
ars.printSchema()
joined.printSchema()

In [ ]:
%pyspark
joined.printSchema()

In [ ]:
%pyspark
joined = joined.join(ars, ars.growerArsId == joined.grower_id[0], 'left_outer')

In [ ]:
%pyspark
joined.printSchema()

In [ ]:
joined.where("id == '0b0f4ba2-dd58-4078-8f77-a091f611851c'").show()

In [ ]:
%pyspark
joined.write.format("json").save("s3n://data-lake-us-east-2-549323063936-validated/timeStampedEventsForFields2")